In [108]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from tqdm import tqdm
import nltk

In [109]:
# Загрузка данных
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [111]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [112]:
# Предобработка текста
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

In [113]:
def preprocess_text(text):
    text = text.lower()  # Привести к нижнему регистру
    tokens = word_tokenize(text)  # Токенизация
    tokens = [token for token in tokens if token.isalpha()]  # Убрать пунктуацию и цифры
    tokens = [token for token in tokens if token not in stop_words]  # Убрать стоп-слова
    tokens = [stemmer.stem(token) for token in tokens]  # Стемминг
    return tokens

In [114]:
train_data['processed_tweet'] = train_data['tweet'].apply(preprocess_text)
test_data['processed_tweet'] = test_data['tweet'].apply(preprocess_text)

In [118]:
# Создание словаря слов
all_words = [word for tokens in train_data['processed_tweet'] for word in tokens]
all_words.extend([word for tokens in test_data['processed_tweet'] for word in tokens])  # Включаем слова из тестовых данных
word_counter = Counter(all_words)
vocab = sorted(word_counter, key=word_counter.get, reverse=True)
word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}  # +1 для резервирования 0 для паддинга
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

In [119]:
# Преобразование текста в последовательности индексов
def encode_text(text):
    return [word_to_idx.get(word, 0) for word in text]  # Используем get для неизвестных слов

In [120]:
train_data['encoded_tweet'] = train_data['processed_tweet'].apply(encode_text)
test_data['encoded_tweet'] = test_data['processed_tweet'].apply(encode_text)

In [121]:
# Создание датасета
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [122]:
X = list(train_data['encoded_tweet'])
y = list(train_data['label'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [123]:
train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [124]:
val_dataset = CustomDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [125]:
# Определение GRU модели
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.gru(embedded)
        last_hidden = output[:, -1, :]
        return self.fc(last_hidden)

In [126]:
# Инициализация модели и других параметров
vocab_size = len(word_to_idx) + 1  # +1 для паддинга
embedding_dim = 100
hidden_dim = 256
output_dim = 2  # Два класса: 0 и 1

In [127]:
model = GRUModel(vocab_size, embedding_dim, hidden_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [128]:
def train_epoch(model, data_loader, criterion, optimizer):
    model.train()
    epoch_loss = 0
    for inputs, labels in tqdm(data_loader, desc="Training"):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [129]:
def evaluate_model(model, data_loader, criterion):
    model.eval()
    epoch_loss = 0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Evaluating"):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            epoch_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_preds += (preds == labels).sum().item()
            total_preds += labels.size(0)
    accuracy = correct_preds / total_preds
    return epoch_loss / len(data_loader), accuracy

In [130]:
num_epochs = 10
best_val_loss = float('inf')

In [132]:
# Функция для добавления паддинга и создания тензоров
def collate_fn(batch):
    data, labels = zip(*batch)
    lengths = [len(seq) for seq in data]
    max_length = max(lengths)

    padded_data = [seq + [0] * (max_length - len(seq)) for seq in data]

    return torch.tensor(padded_data), torch.tensor(labels)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [133]:
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

Evaluating: 100%|██████████| 100/100 [00:01<00:00, 63.15it/s]


Epoch 1/10: Train Loss: 0.2061 | Val Loss: 0.1587 | Val Accuracy: 0.9457


Evaluating: 100%|██████████| 100/100 [00:01<00:00, 63.79it/s]


Epoch 2/10: Train Loss: 0.1249 | Val Loss: 0.1398 | Val Accuracy: 0.9526


Evaluating: 100%|██████████| 100/100 [00:01<00:00, 62.62it/s]


...

Epoch 8/10: Train Loss: 0.0048 | Val Loss: 0.2972 | Val Accuracy: 0.9387


Evaluating: 100%|██████████| 100/100 [00:02<00:00, 46.04it/s]


Epoch 9/10: Train Loss: 0.0060 | Val Loss: 0.2701 | Val Accuracy: 0.9510


Evaluating: 100%|██████████| 100/100 [00:01<00:00, 62.30it/s]

Epoch 10/10: Train Loss: 0.0028 | Val Loss: 0.2849 | Val Accuracy: 0.9481


In [134]:
# Загрузка лучшей модели и предсказание на тестовых данных
best_model = GRUModel(vocab_size, embedding_dim, hidden_dim, output_dim)
best_model.load_state_dict(torch.load('best_model.pth'))
best_model.eval()

GRUModel(
  (embedding): Embedding(38660, 100)
  (gru): GRU(100, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

In [137]:
test_dataset = CustomDataset(list(test_data['encoded_tweet']), [0] * len(test_data))  # Фейковые метки, не используются
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)  # Используем collate_fn

predictions = []
with torch.no_grad():
    for inputs, _ in tqdm(test_loader, desc="Predicting"):
        outputs = best_model(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

Predicting: 100%|██████████| 269/269 [00:06<00:00, 41.08it/s]


In [138]:
# Сохранение предсказаний в файл
test_data['label'] = predictions
test_data[['id', 'label']].to_csv('predictions.csv', index=False)

### Исходя из результатов, можно сделать следующие выводы:
1. Обучение:
  - Значение функции потерь (Train Loss) снижается с каждой эпохой, что говорит о том, что модель учится на обучающих данных.
  - Однако значение функции потерь на валидационных данных (Val Loss) начинает возрастать после нескольких эпох. Это может указывать на переобучение модели.
  - Точность на валидационных данных (Val Accuracy) держится на довольно высоком уровне (в среднем около 95%), что говорит о том, что модель способна хорошо обобщить на новые данные.
2. Предсказания:
  - Лучшая модель, сохраненная в файле 'best_model.pth', имеет следующую архитектуру: Embedding - GRU - Linear. Embedding имеет размер словаря 38660 и размерность векторов 100.
  - В процессе предсказания на тестовых данных модель демонстрирует хорошую скорость (269 батчей обработаны за 6 секунд).

### Помогло улучшить точность нейронной сети:
1. Предобработка текста:
  - Приведение текста к нижнему регистру.
  - Токенизация текста.
  - Удаление пунктуации и цифр.
  - Удаление стоп-слов.
  - Применение стемминга.
2. Включение слов из обучающего и тестового набора данных в словарь.
3. Добавление паддинга к коротким твитам для создания батчей с равной длиной.
4. Создание GRU-модели с Embedding, GRU-слоем и Linear-слоем.
5. Оптимизация и функция потерь:
  - Использование оптимизатора Adam.
  - Использование функции потерь CrossEntropyLoss.
6. Использование DataLoader с collate_fn для обработки разных длин твитов.